<a href="https://colab.research.google.com/github/aktolkyn-masterschool/aktolkyn.p_projects/blob/main/Funnel%20Analysis%20in%20SQL%2C%20Tableau/Funnel_Analysis_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This project aims to analyze the customer funnel of Metrocar, a ride-sharing app (similar to Uber/Lyft), to identify areas for improvement and optimization. I use SQL for data analysis and Tableau for data visualization.

https://public.tableau.com/app/profile/aktolkyn.perritaz/viz/CustomerFunnelAnalysis_carsharingmobileapplication/Story_Funnel_Anlysis


Metrocar’s Funnel:
The customer funnel for Metrocar typically includes the following stages:

STAGE 1: App Download: A user downloads the Metrocar app from the App Store or Google Play Store.
STAGE 2: Signup: The user creates an account in the Metrocar app, including their name, email, phone number, and payment information.
STAGE 3: Request Ride: The user opens the app and requests a ride by entering their pickup location, destination, and ride capacity (2 to 6 riders).
STAGE 4: Driver Acceptance: A nearby driver receives the ride request and accepts the ride.
STAGE 5: Ride: The driver arrives at the pickup location, and the user gets in the car and rides to their destination.
STAGE 6: Payment: After the ride, the user is charged automatically through the app, and a receipt is sent to their email.
STAGE 7: Review: The user is prompted to rate their driver and leave a review of their ride experience.

Similar to other customer funnels, there will be drop-offs at every stage of the funnel, which is why funnel analysis can be helpful in identifying areas for improvement and optimization.

Funnel Analysis with SQL:
1. Define the stages of your funnel
2. Follow the same users through the funnel
3. Calculate Conversion Rates
4. Calculate Drop-off rates

I will follow the below questions to explore the dataset with SQL.

The dataset consists of the following tables:
- app_downloads: contains information about app downloads
- signups: contains information about new user signups
- ride_requests: contains information about rides
- transactions: contains information about financial transactions based on completed rides
- reviews: contains information about driver reviews once rides are completed



**Metrocar’s Funnel**

The customer funnel for Metrocar typically includes the following stages:

1. **App Download:** A user downloads the Metrocar app from the App Store or
Google Play Store.
2. **Signup:** The user creates an account in the Metrocar app, including their name, email, phone number, and payment information.
3. **Request Ride:** The user opens the app and requests a ride by entering their pickup location, destination, and ride capacity (2 to 6 riders).
4. **Driver Acceptance:** A nearby driver receives the ride request and accepts the ride.
5. Ride:The driver arrives at the pickup location, and the user gets in the car and rides to their destination.
6. **Payment:** After the ride, the user is charged automatically through the app, and a receipt is sent to their email.
7. **Review:** The user is prompted to rate their driver and leave a review of their ride experience.

**Analyzing User Journey Across Funnel Stages to Boost Conversion Rates**

Objective:
1. Analyze the customer funnel of a Metrocar, a ride-sharing app company
2. Identify areas for improvement and optimisation

Exploring the Funnel*:
1.     Customer funnel - tracking the flow of users and analysing the conversion rates
2.     Rides funnel - tracking the total number of rides through the funnel and analysing the conversion rates
3.     Analyse the funnel by user age group to identify the target customers to improve the convesion rates

About Metrocar
Metrocar provides a platform that connects riders with drivers through a mobile application. Metrocar acts as an intermediary between riders and drivers, providing a user-friendly platform to connect them and facilitate the ride-hailing process

*Note: PostgreSQL was used for the dataset analysis and Tableau for visualisation and presentation the story

**Customer Funnel: Analyzing User Journey and Conversion Rates **

I will start from Customer FunnelAnalysis to track the User Journey and Conversion Rates





In [ ]:
--Customer Funnel: Analyzing User Journey and Conversion Rates **

I will start from Customer FunnelAnalysis to track the User Journey and Conversion Rates

WITH funnel_num_users as

(SELECT COUNT(DISTINCT app_download_key), 'app_downloads' as user_action
FROM app_downloads
UNION

SELECT COUNT(DISTINCT user_id), 'signed_up' as user_action
FROM signups
UNION

SELECT COUNT (DISTINCT user_id), 'requested_ride' as user_action
FROM ride_requests
UNION

SELECT COUNT (DISTINCT user_id), 'completed_ride' as user_action
FROM ride_requests
WHERE cancel_ts IS NULL
UNION

SELECT COUNT (DISTINCT user_id), 'reviewed' as user_action
FROM reviews)

SELECT user_action, COUNT, lag(count,1) over (ORDER BY count DESC) as lag,
round((count::numeric/lag(count, 1) over (ORDER BY count DESC))*100,1) as drop_off
FROM funnel_num_users



In [ ]:
-- Analyse rides requested & rides completed by age group

WITH t1 AS
(SELECT s.age_range, COUNT (DISTINCT rr.user_id) as users_rides_completed
FROM signups s
LEFT JOIN ride_requests rr ON rr.user_id=s.user_id
GROUP BY s.age_range, rr.cancel_ts
HAVING rr.cancel_ts IS NULL),

t2 as
(SELECT s.age_range, COUNT (DISTINCT rr.user_id) as users_rides_requested
FROM signups s
LEFT JOIN ride_requests rr ON rr.user_id=s.user_id
GROUP BY s.age_range)

SELECT t1.age_range, t1.users_rides_completed, t2.users_rides_requested
FROM t1
JOIN t2 ON t1.age_range = t2.age_range



Analysis of the Customer Funnel:
·     23, 608 total users downloaded the Metrocar app
·      75% of users, who downloaded the app, signed up
·      70% of signups used the app and requested a ride
·      50% of users, who requested a ride, took a ride
·      70% of users, who took a ride, left a review



Main findings of the Customer Funnel Analysis:
·     23, 608 total users downloaded the Metrocar app
·      75% of users, who downloaded the app, signed up
·      70% of signups used the app and requested a ride
·      50% of users, who requested a ride, took a ride
·      70% of users, who took a ride, left a review

**Key Findings of the Customer Funnel Analysis: **
·      Only half of the users who requested a ride, have actually taken a ride while another half have dropped out of the funnel
·     User age group* of 35-44 and 25-34 have the biggest drop out of users from requesting a ride to actually taking a ride

*Note: age group data is not available for all users

Next, I will analyse Rides Funnel to track the rides flow through the app

In [ ]:
--Funnel to follow the user experience and the conversion rates

WITH funnel_rides as

(SELECT COUNT (DISTINCT ride_id), 'rides_requested' as ride_step
FROM ride_requests
UNION

SELECT COUNT (accept_ts), 'rides_driver_accepted' as ride_step
FROM ride_requests
WHERE accept_ts IS NOT NULL
UNION

SELECT COUNT (DISTINCT ride_id), 'rides_completed' as ride_step
FROM ride_requests
WHERE cancel_ts IS NULL
UNION

SELECT count(distinct ride_id), 'rides_paid' as ride_step
FROM transactions
WHERE charge_status = 'Approved'
UNION

SELECT COUNT (DISTINCT review_id), 'reviewed' as ride_step
FROM reviews
UNION

SELECT COUNT (DISTINCT review_id), 'reviewed_3&4&5' as ride_step
FROM reviews
WHERE rating = 4 OR rating = 5 OR rating = 3)

SELECT ride_step, COUNT, lag(count,1) over (ORDER BY count DESC) as lag,
round((count::numeric/lag(count, 1) over (ORDER BY count DESC)),2)*100 as drop_off
FROM funnel_rides


**Analysis of the Rides Funnel:**

·     385,477 total rides requested on the the app
·      64% of requested rides were accepted by a driver
·      90% of accepted rides were completed for customer
·      95% of completed rides were paid
·      73% of paid rides were reviewed by customer
·     60% of reviewed rides were evaluated as 'Excellent', 'Good' and 'Average'


**Key Findings of the Rides Funnel Analysis: **

·      Only half of the users who requested a ride, have actually taken a ride and another half have dropped out of the funnel
·     Need to look into how to improve 70% & 75% conversion rates on other steps







** Funnel Areas For Improvement and Optimisation  **

· Customer Funnel Analysis indicates that the conversion of users from requesting a ride to taking it needs to be improved


· User age groups of 35-44 and 25-34 deserve particular attention as these are the target groups with the highest number of users dropped out of the funnel


· 'Acceptance of rides by a driver' needs to be investigated and optimised as it is the main reason of lower conversion rate at the stage of requesting a ride to completing it


· Quality of rides is another critical area for improvement
















